# Chapter 6: Reading and Writing Data

In [ ]:
import polars as pl
pl.show_versions()  # The book is built with Polars version 1.13.0

In [ ]:
import polars as pl

## Format Overview

## Reading CSV Files

In [ ]:
!cat data/penguins.csv

In [ ]:
penguins = pl.read_csv("data/penguins.csv")
penguins

## Parsing Missing Values Correctly

In [ ]:
penguins = pl.read_csv("data/penguins.csv", null_values="NA")
penguins

In [ ]:
penguins.null_count().transpose(  
    include_header=True, column_names=["null_count"]
)

## Reading Files with Encodings Other than UTF-8

In [ ]:
# This raises a ComputeError:
# pl.read_csv("data/directors.csv")

In [ ]:
pl.read_csv("data/directors.csv", encoding="EUC-CN")

In [ ]:
import chardet


def detect_encoding(filename: str) -> str:
    """Return the most probable character encoding for a file."""

    with open(filename, "rb") as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        return result["encoding"]


detect_encoding("data/directors.csv")

In [ ]:
pl.read_csv("data/directors.csv", encoding="EUC-JP")

## Reading Excel Spreadsheets

In [ ]:
songs = pl.read_excel("data/top2000-2023.xlsx")  
songs

## Working with Multiple Files

In [ ]:
pl.read_csv("data/stock/nvda/201?.csv")

In [ ]:
all_stocks = pl.read_csv("data/stock/**/*.csv")
all_stocks

In [ ]:
import calendar

filenames = [
    f"data/stock/asml/{year}.csv"
    for year in range(1999, 2024)
    if calendar.isleap(year)
]

filenames

In [ ]:
pl.concat(pl.read_csv(f) for f in filenames)

## Reading Parquet

In [ ]:
%time
trips = pl.read_parquet("data/taxi/yellow_tripdata_*.parquet")
trips

## Reading JSON and NDJSON

### JSON

In [ ]:
!cat data/pokedex.json

In [ ]:
pokedex = pl.read_json("data/pokedex.json")
pokedex

In [ ]:
(
    pokedex.explode("pokemon")
    .unnest("pokemon")
    .select("id", "name", "type", "height", "weight")
)

### NDJSON

In [ ]:
!cat data/wikimedia.ndjson

In [ ]:
from json import loads
from pprint import pprint

with open("data/wikimedia.ndjson") as f:
    pprint(loads(f.readline()))

In [ ]:
wikimedia = pl.read_ndjson("data/wikimedia.ndjson")
wikimedia

In [ ]:
(
    wikimedia.rename({"id": "edit_id"})
    .unnest("meta")
    .select("timestamp", "title", "user", "comment")
)

## Other File Formats

In [ ]:
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_Latin_abbreviations"
pl.from_pandas(pd.read_html(url)[0])  

## Querying Databases

In [ ]:
pl.read_database_uri(
    query="""
    SELECT
        f.film_id,
        f.title,
        c.name AS category,
        f.rating,
        f.length / 60.0 AS length
    FROM
        film AS f,
        film_category AS fc,
        category AS c
    WHERE
        fc.film_id = f.film_id
        AND fc.category_id = c.category_id
    LIMIT 10
    """,
    uri="sqlite:::data/sakila.db",
)

In [ ]:
db = "sqlite:::data/sakila.db"
films = pl.read_database_uri("SELECT * FROM film", db)
film_categories = pl.read_database_uri("SELECT * FROM film_category", db)
categories = pl.read_database_uri("SELECT * FROM category", db)

(
    films.join(film_categories, on="film_id", suffix="_fc")
    .join(categories, on="category_id", suffix="_c")
    .select(
        "film_id",
        "title",
        pl.col("name").alias("category"),
        "rating",
        pl.col("length") / 60,
    )
    .limit(10)
)

## Writing Data

### CSV Format

In [ ]:
all_stocks.write_csv("data/all_stocks.csv")

### Excel Format

In [ ]:
all_stocks.write_excel("data/all_stocks.xlsx")

### Parquet Format

In [ ]:
all_stocks.write_parquet("data/all_stocks.parquet")

### Other Considerations

## Takeaways